Final Project - AWS ETL Data Streaming 
We decided to create a Data Pipeline on AWS that could facilitate the process of building a Natural Language Processing tool. The tool will utilize a simple machine learning algorithm or one/two layer neural network to train a topic modeler or sentiment analyzer or a recommender. The technologies (tentative) that will be used in this project

* Terraform to create template for cloud managment and configuration 
* Lambda and SAM to streamline a simple ETL data pipeline that extracts, processes, and load the data from Twitter API(tentative)
* Eventbridge(Cloudwatch events) to log and monitor state changes 
* S3 Bucket to store the raw retrieved data (potentially utilize DynamoDB to store the parsed and processed data for future analytics)
* Sagemaker to create a notebook for hosting the ML project code, fine-tuning  parameters, pipelining and storing the models
* Pandas to ingest and wrangle the data and preprocess it for ML training
* EC2 to create a VM that hosts the workflow or provide workers for training if needed 
* (Tentative) DynamoDB to store a structured version of the output data for future reference 

Terraform templapte 
to create  and manage an aws infrastructure 


Run test code for Terraform templates

In [ ]:
import os
os.chdir('/Users/shiyishen/nlp/aws-data-engineering/aws-data-pipeline/cdk-tf/')


### Install dependencies and packages for the project

In [ ]:
!pip install boto3
!pip install awswrangler
!pip install tweepy
!pip install numpy


### Data streaming with Lambda from Twitter API

In [ ]:
# TODO
# Twitter pulling class 

In [ ]:
# Twitter processing class 


In [ ]:
# Twitter loading class 

### Data Wrangling and transformation with Pandas

### Data loading to S3

### Sagemaker Machine Learning LDA Algorithm on EC2

In [ ]:
#data loading

In [ ]:
#sagemaker imports 
#